In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits

import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

# Open FITS file

In [ ]:
fname = "../hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdu_list = fits.open(fname)
hdu_list.info()

In [ ]:
#read in the image data and close the file using "getdata() shortcut"
image_data = fits.getdata(fname)
image_data = image_data.byteswap(inplace=True).newbyteorder()
print(type(image_data))
print(image_data.shape)

In [ ]:
#close the FITS file
hdu_list.close()

In [ ]:
#show the image
mask = image_data==0
m, s = np.mean(image_data[~mask]), np.std(image_data[~mask])

plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower',)
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/image_data.png')

# background subtraction

In [ ]:
#measure a spatially varying background on the image
#mask pixels
bkg = sep.Background(image_data, mask=mask, bw=64, bh=64, fw=3, fh=3)

In [ ]:
#get a "global" mean and noise of the image background
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate backgound as a 2D array, same size as the original image
bkg_image = np.array(bkg)

In [ ]:
#show background noise
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/bkg_image.png')

In [ ]:
#evaluate the backgound noise as a 2D array, same size as the original image
bkg_rms = bkg.rms()

In [ ]:
#show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/bkg_rms.png')

In [ ]:
#subtract the background
data_sub = image_data - bkg

# Object Detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
len(objects)
#detected ~3500 objects

In [ ]:
from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
mask = data_sub==0
m, s = np.mean(data_sub[~mask]), np.std(data_sub[~mask])
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar(im)
plt.savefig('/Users/labuser/Desktop/data_sub.png')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), 
                width=6*objects['a'][i], 
                height=6*objects['b'][i], 
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
data_sub

In [ ]:
#available fields
objects.dtype.names

# Aperture photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
#show the first 10 objects results
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

# Histogram fluxes


In [ ]:
#convert to AB magnitudes
header = fits.getheader(fname)


x = header["PHOTFLAM"]
y = header["PHOTPLAM"]

#calculate zero pt 
zp_ambag = -2.5*np.log10(x) - 5*np.log10(y) - 2.408


abmags = -2.5*np.log10(flux) + zp_ambag


#plot AB magnitudes
plt.hist(abmags);
